In [1]:
# can you edit this tom?

#i have been able to edit from VScode

In [2]:
#I am now updating a copy

In [3]:
# yes seems like it's working :-)
#  hopefully it loads back up...

In [4]:
# a modified code from first_branch

In [5]:
# yes seems like it's working :-)
#  hopefully it loads back up...

In [6]:
# update from maria to check that this is working - can everyone see this?
# Anna - I can and edit in VS